In [1]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from scipy.optimize import *
import matplotlib.pyplot as plt

Cuda is not available!
logicqubit version 1.5.8


In [2]:
gates = Gates(1)

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [3]:
II = ID.kron(ID)
XX = X.kron(X)
YY = Y.kron(Y)
ZZ = Z.kron(Z)
IZ = ID.kron(Z)
ZI = Z.kron(ID)
sig_ss = np.kron([1, -1], [1, -1])
sig_is = np.kron([1, 1], [1, -1])
sig_si = np.kron([1, -1], [1, 1])
sig_is
ZZ.get()

array([[ 1,  0,  0,  0],
       [ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 0,  0,  0,  1]])

In [4]:
H = II*2 + YY + IZ*3 - ZI + ZZ*2

min(scipy.linalg.eig(H.get())[0])

(-4.123105625617661+0j)

In [5]:
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_2q(q1, q2, params):
    q2.CNOT(q1)
    q1.RY(params[0])
    q2.RY(params[1])
    q1.CNOT(q2)
    q1.RY(params[2])
    q2.RY(params[3])
    q2.CNOT(q1)
    q1.RY(params[4])
    q2.RY(params[5])

In [6]:
def expectation_2q(params):
    logicQuBit  = LogicQuBit(2)
    q1 = Qubit()
    q2 = Qubit()

    ansatz_2q(q1,q2,params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

minimum = minimize(expectation_2q, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:586: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[k] = func(sim[k])
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:618: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:611: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxe
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:614: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:627: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxc
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:637: ComplexWarning: Casting comp

 final_simplex: (array([[-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761],
       [-1.08588037, -1.20307451,  1.5931768 ,  0.98904206, -0.43267192,
         0.89020761]]), array([-4.12310563, -4.12310563, -4.12310563, -4.12310563, -4.12310563,
       -4.12310563, -4.12310563]))
           fun: -4.123105625617663
       message: 'Optimization terminated successfully.'
          nfev: 899
           nit: 518
        status: 0
       success: True
             x: array([-1.08588037, -1.

/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:644: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[j] = func(sim[j])


In [7]:
def expectation_value(measurements, base = np.array([1,-1,-1,1])):
    probabilities = np.array(measurements)
    expectation = np.sum(base * probabilities)
    return expectation

def sigma_xx(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_yy(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_zz(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
          
    result = logicQuBit.Measure([q1,q2])
    zz = expectation_value(result)
    iz = expectation_value(result, sig_is) # [zz, iz] = 0
    zi = expectation_value(result, sig_si) # [zz, zi] = 0
    return zz, iz, zi

def expectation_energy(params):
    #xx =  sigma_xx(params)
    yy =  sigma_yy(params)
    zz, iz, zi =  sigma_zz(params)

    result = 2 + yy + 3*iz - zi + 2*zz
    return result

In [8]:
minimum = minimize(expectation_energy, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882],
       [ 1.71336166, -3.14720546, -0.97839616,  2.16971816, -2.76042898,
         1.02473882]]), array([-4.12310563, -4.12310563, -4.12310563, -4.12310563, -4.12310563,
       -4.12310563, -4.12310563]))
           fun: -4.123105625617663
       message: 'Optimization terminated successfully.'
          nfev: 991
           nit: 574
        status: 0
       success: True
             x: array([ 1.71336166, -3.

In [14]:
def gradient(params, evaluate, shift = pi/2):
    n_params = params.shape[0]
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [15]:
params = np.random.uniform(-np.pi, np.pi, 6)
last_params = np.zeros(6)

In [16]:
lr = 0.1
err = 1
while err > 1e-3:
    grad = gradient(params, expectation_energy, pi/1.2)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

/tmp/ipykernel_10875/2399136664.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


6.482663126567106
0.38223934285331607
0.24488311284400144
0.15336235431677614
0.09703052432549342
0.06171935514192828
0.039443704313149364
0.025282440007640683
0.01623888046263866
0.010444314063609794
0.006723457691249157
0.004330708948179662
0.0027905545833459833
0.0017985774254206666
0.0011594101206759966
0.0007474632522979086


In [17]:
expectation_energy(params)

(-4.123104878258455+0j)